In [1]:
import pygko
import numpy as np
import scipy

In [2]:
# dir(pygko)

In [3]:
# dir(pygko.matrix)

In [4]:
# help(pygko.matrix.Dense)

# Create Executors

In [5]:
ref = pygko.ReferenceExecutor()
omp = pygko.OmpExecutor()
try:
    device = pygko.allocation_mode.device 
    cuda = pygko.CudaExecutor(0, omp, False, device)
    print(cuda)
    executor = cuda
except Exception as e:
    executor = ref
    print(e)

# Create Dense from numpy array

In [6]:
numpy_array = np.array([1.,2.,3.,4.,5.])
dense = pygko.matrix.Dense(cuda, numpy_array)
scalar =  pygko.matrix.Dense(cuda,  np.array([2.]))
dense.scale(scalar)
np.array(dense.copy_to_host())

array([ 2.,  4.,  6.,  8., 10.])

# Read from disk

In [6]:
path = "/hkfs/home/project/hk-project-test-fine/eq4036/data/code/ginkgo/examples/simple-solver/data/"

scipy_b =scipy.io.mmread(path + "b.mtx")
b = pygko.matrix.Dense(executor, scipy_b)

scipy_x = scipy.io.mmread(path + "x0.mtx")
x = pygko.matrix.Dense(executor, scipy_x)

A = pygko.read_coo(path + "A.mtx", executor)

In [7]:
print(type(b),type(x), type(A))

<class 'pygko.matrix.Dense'> <class 'pygko.matrix.Dense'> <class 'pygko.matrix.Coo'>


# Create Stopping Criterion

In [8]:
stop = pygko.stop.Iteration(executor, 1000)
preconditioner = pygko.preconditioner.Jacobi(executor, 16)

# Create Solver

In [9]:
cg = pygko.solver.Cg(executor, A, [stop, preconditioner])
cg.apply(b, x)

In [10]:
np_zero = np.array([0.0])
np_one = np.array([1.0])
np_neg_one = np.array([-1.0])

one = pygko.matrix.Dense(executor, np_one)
neg_one = pygko.matrix.Dense(executor, np_neg_one)
res = pygko.matrix.Dense(executor, np_zero)

In [11]:
A.apply(one, x, neg_one, b)

pygko.matrix.Coo object

In [12]:
b.compute_norm2(res)
print(np.array(res.copy_to_host(),copy=True))

[7.13053648e-16]
